# Assignment 2 - MATH60837A
William Bourque - 11359215

Frederic Pelletier - 11359258

## installing dependencies (only needed once)

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install pandas
%pip install statsmodels
%pip install arch

## importing dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf, q_stat
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import chi2
from arch import arch_model
from scipy.optimize import minimize, LinearConstraint
import warnings
warnings.filterwarnings("ignore")

## Loading the data

In [ ]:
df = pd.read_csv('./data.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df['log_HP'] = np.log(df['QCAR628BIS'])
df['diff_HP'] = df['log_HP'].diff()
df = df.dropna()
yt = df['diff_HP']
zt = df['log_HP']
df.head()

# Part1.Model Selection

### 1.a) Estimate all 4 models by maximum likelihood, report the estimation results, and verify if the stationarity conditions and invertibility conditions are satisfied

In [ ]:
def test_stationarity(model: ARIMA):
    p,_,q = model.specification.order
    r = max(p,q)
    phi_matrix = np.zeros((r,r))
    phis = model.arparams
    phi_matrix[0,:] = np.pad(model.arparams, (0, r - len(phis)), mode='constant')
    np.fill_diagonal(phi_matrix[1:p,:p],1)
    eigenvalues = np.linalg.eigvals(phi_matrix)
    print(phi_matrix)
    return np.all(np.abs(eigenvalues) < 1)

def test_invertibility(model:ARIMA):
    p,_,q = model.specification.order
    r = max(p,q)
    theta_matrix = np.zeros((r,r))
    thetas = model.maparams
    theta_matrix[0,:] = np.pad(thetas, (0,r - len(thetas)), mode='constant')
    np.fill_diagonal(theta_matrix[1:q,:q],1)
    eigenvalues = np.linalg.eigvals(theta_matrix)
    print(theta_matrix)
    return np.all(np.abs(eigenvalues) < 1)

In [ ]:
#since we will be testing for stationarity and invertibility, we do not force the
# algorithm to ensure either of them
ar4 = ARIMA(yt, order=(4,0,0), enforce_stationarity=False, enforce_invertibility=False).fit()
arma41 = ARIMA(yt, order=(4,0,1),enforce_stationarity=False, enforce_invertibility=False).fit()
arma42 = ARIMA(yt, order=(4,0,2),enforce_stationarity=False, enforce_invertibility=False).fit()


for name,model in {'ar(4)': ar4,'arma(4,1)': arma41,'arma(4,2)': arma42}.items():
    stationary = test_stationarity(model)
    invertible = test_invertibility(model)
    print(model.summary())
    print(f'{name} is {"stationary" if stationary else "not stationary"}')
    print(f'{name} is {"invertible" if invertible else "not invertible"}')
    

### Interpretation
All models are both stationary and invertible since the eigenvalues of their respective phi and theta matrices all have modules less than 1. However, for all models, some parameters 
do not have a p-value < 0.05, meaning there is no strong statistical evidence that the parameter should be different than 0

### 1.b) Select a model among the 3 using the BIC criterion. Perform likelihood ratio tests (using a 5% significance level) to discriminate between that model and possible alternatives among the remaining 2. Evaluate the white noise hypothesis for the residuals to justify the selection of a final model.

In [ ]:
# We evaluate the BIC creterion for each model
print(f'BIC for AR(4): {ar4.bic}')
print(f'BIC for ARMA(4,1): {arma41.bic}')
print(f'BIC for ARMA(4,2): {arma42.bic}')


In [ ]:
# We first define a function to help us perform the test
def likelihood_ratio_test(l1,l0,k, alpha):
    statistic = 2*(l1 - l0)
    critical_value = chi2.ppf(1 - alpha, k)
    print(f'{statistic=}, {critical_value=}')
    print('p-value:',1 - chi2.cdf(statistic, k)) # P(chi2 > statistic)
    return statistic > critical_value

In [ ]:
model_from_first_test = None
model_from_second_test = None
can_reject_null = likelihood_ratio_test(arma41.llf, ar4.llf, 1, 0.05)
if can_reject_null:
    print('we can reject the null hypothesis, meaning we pick the AMRA(4,1)')
    model_from_first_test = arma41
else:
    print('we cannot reject the null hypothesis, meaning we pick the AR(1)')
    model_from_first_test = ar4

can_reject_null = likelihood_ratio_test(arma42.llf, model_from_first_test.llf, 1, 0.05)
if can_reject_null:
    print('we can reject the null hypothesis, meaning we pick the ARMA(4,2)')
    model_from_second_test = arma42
else:
    print('we cannot reject the null hypothesis, meaning we pick the ARMA(4,1)')
    model_from_second_test = arma41

selected_model = model_from_second_test

In [ ]:
# We perform the Lyung-Box test to make sure the residuals of the model are white noise
residuals = selected_model.resid
lags = range(1, 19)  # Creat a vector for lags 1 to 18

# Perform the Ljung-Box Q-test
lbq_result = acorr_ljungbox(residuals, lags=lags, return_df=True)

p_values = lbq_result['lb_pvalue']
test_statistics = lbq_result['lb_stat']
h1 = (p_values < 0.05).astype(int)  # Binary decision rule (1 = reject null hypothesis)

# Print the results
print("Decision Rule (h1):", h1.values)
print("P-Values:", p_values.values)
print("Test Statistics:", test_statistics.values)
acf(residuals,nlags=20)

if h1.mean() == 0:
    chosen_model = selected_model

### Interpretation 
Using the BIC creterion the selected model is the ARMA(4,1), because its value of -898.96 was the smallest among the three models. To verify the potency of this result we perform the likelihood ratio test. The first test uses the AR(4) as null hypothesis angainst the ARMA(4,1). The p-value is smaller than the chosen alpha meaning we reject the null and select the ARMA(4,1). The second test uses the selected model from the first test, ARMA(4,1) as the null against the ARMA(4,2). The p-value is greater than the chosen alpha meaning we cannot reject the null and select once again the ARMA(4,1). To verify the white noise hypothesis for the residuals we perform the Lyung-Box test. The test is concluent, we cannot reject the null for the first 18 lags, it suggests that the residuals are white noise and the model can bu useful.

# Part 2. Dynamic Response and forecasting

### 2.a) For the selected model, evaluate the dynamic response for an horizon of 10 periods following a positive shock of size sigma = 1.15 occuring at the first period of the horizon, and explain what happens.

In [ ]:
print(chosen_model.params)
plt.plot(chosen_model.impulse_responses(10,[1.15]), label='response')
plt.xlabel('Periods')
plt.ylabel('Response value')
plt.title('Dynamic response following a shock of 1.15')
plt.legend()

### Interpretation 

We can see that following the shock, the dynamic response quickly fades to 0. However, it is not strictly decreasing, which makes sense given the polynomial nature of the dynamic response equation.

### 2.b) For the selected model, evaluate the dynamic response for an horizon of 10 periods following a positive shock of size occurs during 3 consecutive periods of the horizon (t, t+1, t+2). Plot the impulse response function, showing the dynamic response for an horizon of 20 periods following these shocks, and explain what happens.

In [ ]:
#we have to use the definition with psi_0 = psi_1 = psi_2 = 1.15
n_periods = 20
shocks = [1.15,1.15,1.15]
total_n_periods = n_periods + len(shocks)
responses = np.append(shocks,[0] * n_periods)
p = len(chosen_model.arparams)
q = len(chosen_model.maparams)
for i in range(len(shocks), total_n_periods):
    p_indices = [i - j for j in range(1,p+1)]
    q_indices = [i - j for j in range(1,q+1)]
    prev_shocks = [responses[pi] if pi >= 0 else 0 for pi in p_indices]
    prev_noises = [1 if qi == 0 else 0 for qi in q_indices]
    responses[i] = np.dot(prev_shocks,chosen_model.arparams) - np.dot(prev_noises, chosen_model.maparams) + (1 if i == 0 else 0)
plt.plot(responses[3:],  label="Response of y to 3 Consecutive Shocks")

# Plot impulse responses
plt.axhline(y=0, linestyle="--", color="black", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Response")
plt.title("Impulse Response of ARMA(4,1) with 3 Consecutive Shocks of 1.15")
plt.legend()
plt.show()

### Interpretation

We can see the the dynamic response takes a longer time before reaching 0, oscillating up and down the 0 line. This is due to the fact that the high-valued shocks, i.e. those with value 1.15, get picked up longer in the recursion, increasing the value of several of the next iterations.

### 2.c) Forecasting: Split the sample into a training sample and a holdout sample. The holdout sample should consist of the last 34 observations 

 - Re-estimate the selected model using only the training sample
 - Calculate the 1-step ahead forecasts for the stationarized series
 - Plot these forecasts along with the actual series on the same graph for the period covered by the hold-out sample
 - Now calculate and plot the level (as opposed to first difference) of the variable zt against its 1-step ahead forecasts from the 2 models and from the naive forecast E(zt+1) = zt

In [ ]:
training_set = yt[:-34]
test_set = yt[-34:] #last 34 data points
forecast_model_init = ARIMA(training_set, order=(4,0,1),enforce_stationarity=False, enforce_invertibility=False).fit()

initial_chunk = test_set.iloc[:4]
forecast_model = forecast_model_init.extend(initial_chunk)


y_forecast_values = []
forecast_dates = test_set.index[4:]  # Get the dates for forecasts

for i in range(4, len(test_set)):
    next_value = test_set.iloc[i]  

    # Generate 1-step-ahead forecast
    forecast_value = forecast_model.forecast(steps=1)[0]
    y_forecast_values.append(forecast_value)

    # Extend the model with the new observed value
    forecast_model = forecast_model.extend([next_value])

# Convert forecasts into a Pandas DataFrame
y_forecast_df = pd.DataFrame({"Forecast": y_forecast_values}, index=forecast_dates)

# Print results
plt.plot(y_forecast_df, label='forecast of yt')
plt.plot(test_set[4:], label='yt')
plt.legend()
plt.title('Forecasted value of yt vs yt')
plt.ylabel('yt')
plt.xlabel('date')
plt.show()

In [ ]:
z_training_set = zt[:-34]
z_test_set = zt[-34:]

z_forecast_model_init = ARIMA(z_training_set, order=(4,0,1),enforce_stationarity=False, enforce_invertibility=False).fit()

initial_chunk = np.asarray(z_test_set.iloc[:4])
z_forecast_model = z_forecast_model_init.extend(initial_chunk)

z_forecast_values = []

forecast_dates = z_test_set.index[4:]  # Get the dates for forecasts

for i in range(4, len(z_test_set)):
    next_value = z_test_set.iloc[i]  

    # Generate 1-step-ahead forecast

    z_forecast_value = z_forecast_model.forecast(steps=1)[0]
    z_forecast_values.append(z_forecast_value)


    # Extend the model with the new observed value
    z_forecast_model = forecast_model.extend([next_value])
# Convert forecasts into a Pandas DataFrame
z_forecast_df = pd.DataFrame({"Forecast": z_forecast_values}, index=forecast_dates)


plt.plot(z_forecast_df, label='forecast of zt with z-model')
plt.plot(z_test_set[4:], label='zt')
plt.plot(z_test_set.shift(1).dropna()[3:], label='forecast of zt with dummy model')
plt.plot()
plt.legend()
plt.title('Forecasted value of zt vs zt')
plt.ylabel('zt')
plt.xlabel('date')
plt.show()

### 2.d) Report the Mean-Squared Error for the selected model using only the hold-out sample of 34 observations to evaluate the forecast performance. Compare these statistics to those from the naive forecast. What model should we use?

In [ ]:
mse_model = ((test_set[4:] - y_forecast_values) ** 2).mean()
mse_naive = ((test_set[4:] - test_set.shift(1).dropna()[3:]) ** 2).mean()
print(mse_model, mse_naive)

# Part 3. Further improvements by modelling the conditionnal variance

### 3.a) Plot the autocorrelation and partial autocorrelation functions of y_t squared. What can you conclude? 

In [ ]:
yt_squared = pow(yt,2)
acf_vector = acf(yt_squared, nlags = 20)
pacf_vector = pacf(yt_squared, nlags = 20)

plt.plot(acf_vector, label = 'Autocorrelation')
plt.plot(pacf_vector, label = 'Partial autocorrelation')
plt.xlabel('Lags')
plt.ylabel('Correlation')
plt.title('Autocorrelation and Partial Autocorrelation')
plt.legend()
plt.show()


### Interpretation

### 3.b) Estimate the ARCH(1) and GARCH(1,1) versions of the model selected in part 1 using only the training sample, and report the estimates.

In [ ]:
# unfortunately, there are no python packages supporting ARMA - GARCH models, so we need to obtain the parameters 
# by optimizing the log-likelihood function manually

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

def arma_garch_loglike(params, y, p, q, r, s):

    n = len(y)
    
    c = params[0]  
    phi = params[1:p+1]  
    theta = params[p+1:p+q+1]  
    alpha0 = params[p+q+1]  
    alpha = params[p+q+2:p+q+2+r]  
    beta = params[p+q+2+r:p+q+2+r+s] 

    ht = np.ones(n) * np.var(y)
    ut = np.zeros(n)  

    
    for t in range(max(p, q, r, s), n):
        # ARMA(p,q) equation
        arma_mean = c
        if p > 0:
            arma_mean += np.dot(phi, y[t-p:t])  # AR terms
        if q > 0:
            arma_mean += np.dot(theta, ut[t-q:t])  # MA terms
        
        # Compute residual
        ut[t] = y[t] - arma_mean
        
        # Compute GARCH(r,s) variance equation
        ht[t] = alpha0
        if r > 0:
            ht[t] += np.dot(alpha, ut[t-r:t]**2)  # ARCH terms
        if s > 0:
            ht[t] += np.dot(beta, ht[t-s:t])  # GARCH terms
        
        


    # Compute log-likelihood, dropping the constant term
    loglike = -0.5 * np.sum(np.log(ht[max(p, q, r, s):]) + (ut[max(p, q, r, s):]**2 / ht[max(p, q, r, s):]))
    
    return -loglike  # We minimize negative log-likelihood, which is mathematically equivalent to maximing log-likelihood, to avoid overflow

def estimate_arma_garch(y, p, q, r, s):

    # Parameter estimation

    initial_params = [forecast_model_init.params['const'],*forecast_model_init.arparams, *forecast_model_init.maparams, *np.random.uniform(0,1,1), *np.random.uniform(0,1,r), *np.random.uniform(0,1,s)]
    

    #        arma(p,q) bounds          alpha0 bound  alpha bound  beta bound
    bounds = [(None,None)] * (p+q+1) + [(1e-8,None)] + [(0,1)] * r + [(0,1)] * s
    constraints = [0] * (p+q+1) + [0] + [1] * r + [1] * s #to ensure 0 < alpha + beta < 1
    result = minimize(arma_garch_loglike, initial_params, args=(y.values, p, q, r, s), bounds=bounds, constraints=LinearConstraint(constraints,0,1)) 

    print(result.success, result.message)
    # Extract optimized parameters
    estimated_params = result.x
    return {
        "c": estimated_params[0],
        "phi": estimated_params[1:p+1],
        "theta": estimated_params[p+1:p+q+1],
        "alpha0": estimated_params[p+q+1],
        "alpha": estimated_params[p+q+2:p+q+2+r],
        "beta": estimated_params[p+q+2+r:p+q+2+r+s],
        "log_likelihood": -result.fun
    }

# the optimization algorithm tends to hit a local minima from time to time, so we will do the 
# process 10 times and mix the max value for both models
arch_model_fit = estimate_arma_garch(training_set,4,1,1,0)
garch_model_fit = estimate_arma_garch(training_set,4,1,1,1)

for i in range(9):
    curr_arch_model_fit = estimate_arma_garch(training_set,4,1,1,0)
    if curr_arch_model_fit['log_likelihood'] > arch_model_fit['log_likelihood']:
        arch_model_fit = curr_arch_model_fit
    curr_garch_model_fit = estimate_arma_garch(training_set,4,1,1,1)
    if curr_garch_model_fit['log_likelihood'] > garch_model_fit['log_likelihood']:
        garch_model_fit = curr_garch_model_fit

print('arch_model:', arch_model_fit)
print('garch_model_fit', garch_model_fit)


### Interpretation

### 3.c) Perform likelihood ratio tests to select one model among the original ARMA model selected in part 1, its ARCH(1) alternative, and its GARCH(1,1) alternative.

In [ ]:
model_from_llr = None
can_reject_null = likelihood_ratio_test(garch_model_fit['log_likelihood'], arch_model_fit['log_likelihood'], 1, 0.05)
if can_reject_null:
    print('we can reject the null hypothesis, meaning we select the GARCH(1,1)')
    model_from_llr = garch_model_fit
else:
    print('we cannot reject the null hypothesis, meaning we pick the ARCH(1)')
    model_from_llr = arch_model_fit

can_reject_null = likelihood_ratio_test(model_from_llr['log_likelihood'],forecast_model_init.llf,2,0.05 )
if can_reject_null:
    print('we can reject the null hypothesis, meaning we select the ARMA(4,1) - GARCH(1,1)')
else:
    print('we cannot reject the null hypothesis, meaning we pick the ARMA(4,1)')
    

### Interpretation

### 3.d) Perform a specification test on the residuals of the ARCH(1) and GARCH(1,1) components. What is the best model for the conditional variance?

In [ ]:
def arma_garch_specification_test(params, y, p, q, r, s):

    n = len(y)
    
    c = params['c']  
    phi = params['phi']  
    theta = params['theta']  
    alpha0 = params['alpha0']  
    alpha = params['alpha']  
    beta = params['beta'] 

    ht = np.ones(n) * np.var(y)
    ut = np.zeros(n)  
    for t in range(max(p, q, r, s), n):
        # ARMA(p,q) equation
        arma_mean = c
        if p > 0:
            arma_mean += np.dot(phi, y[t-p:t])  # AR terms
        if q > 0:
            arma_mean += np.dot(theta, ut[t-q:t])  # MA terms
        
        # Compute residual
        ut[t] = y[t] - arma_mean
        
        # Compute GARCH(r,s) variance equation
        ht[t] = alpha0
        if r > 0:
            ht[t] += np.dot(alpha, ut[t-r:t]**2)  # ARCH terms
        if s > 0:
            ht[t] += np.dot(beta, ht[t-s:t])  # GARCH terms

    return ut**2 / ht

zt_arch = arma_garch_specification_test(arch_model_fit, training_set, 4,1,1,0)
zt_garch = arma_garch_specification_test(garch_model_fit, training_set, 4,1,1,1)

lags = range(1, 19)  # Creat a vector for lags 1 to 18

# Perform the Ljung-Box Q-test
lbq_result = acorr_ljungbox(zt_arch[4:], lags=lags, return_df=True)

p_values = lbq_result['lb_pvalue']
test_statistics = lbq_result['lb_stat']
h1 = (p_values < 0.05).astype(int)  # Binary decision rule (1 = reject null hypothesis)

# Print the results
print("Decision Rule (h1):", h1.values)
print("P-Values:", p_values.values)
print("Test Statistics:", test_statistics.values)
plt.plot(acf(zt_arch[4:],nlags=20), label='acf(zt_arch)')


# Perform the Ljung-Box Q-test
lbq_result = acorr_ljungbox(zt_garch[4:], lags=lags, return_df=True)

p_values = lbq_result['lb_pvalue']
test_statistics = lbq_result['lb_stat']
h1 = (p_values < 0.05).astype(int)  # Binary decision rule (1 = reject null hypothesis)

# Print the results
print("Decision Rule (h1):", h1.values)
print("P-Values:", p_values.values)
print("Test Statistics:", test_statistics.values)
plt.plot(acf(zt_garch[4:],nlags=20), label='acf(zt_garch)')

plt.legend()

### Interpretation

### 3.e) For these two new models and the model studied in Part 2, plot the impulse response functions of the second case described in Part 2. How different are they?

In [ ]:
#we have to use the definition with psi_0 = psi_1 = psi_2 = 1.15
n_periods = 20
shocks = [1.15,1.15,1.15]

def impulse_response_arma_garch(params,n_periods,shocks):
    total_n_periods = n_periods + len(shocks)
    responses = np.append(shocks,[0] * n_periods)
    p = len(params['phi'])
    q = len(params['theta'])
    for i in range(len(shocks), total_n_periods):
        p_indices = [i - j for j in range(1,p+1)]
        q_indices = [i - j for j in range(1,q+1)]
        prev_shocks = [responses[pi] if pi >= 0 else 0 for pi in p_indices]
        prev_noises = [1 if qi == 0 else 0 for qi in q_indices]
        responses[i] = np.dot(prev_shocks,params['phi']) - np.dot(prev_noises, params['theta']) + (1 if i == 0 else 0)
    return responses

arch_responses = impulse_response_arma_garch(arch_model_fit,20,shocks)
garch_responses = impulse_response_arma_garch(garch_model_fit, 20, shocks)
plt.plot(arch_responses[3:],  label="Dynamic response for ARMA(4,1) - ARCH(1)")
plt.plot(garch_responses[3:], label="Dynamic response for ARMA(4,1) - GARCH(1,1)")
plt.plot(responses[3:], label="Dynamic response for ARMA(4,1)")
# Plot impulse responses
plt.axhline(y=0, linestyle="--", color="black", linewidth=1)
plt.xlabel("Time")
plt.ylabel("Response")
plt.title("Impulse Response of ARMA(4,1) with 3 Consecutive Shocks of 1.15")
plt.legend()
plt.show()

### Interpretation

### 3.f)Like in Part 2, plot the 1-step ahead forecasts for for the period covered by the holdout sample. Compare them with those obtained in Part 2. Are they different from before? If so, why?

In [ ]:
def arma_garch_forecast(params, y, p,q,r,s):
    n = len(y)
    forecast_dates = y.index[max(p,q,r,s):]
    c = params['c']  
    phi = params['phi']  
    theta = params['theta']  
    ut = np.zeros(n)
    forecasts = []  
    for t in range(max(p, q, r, s), n):
        # ARMA(p,q) equation
        forecast = c
        if p > 0:
            forecast += np.dot(phi, y[t-p:t])  # AR terms
        if q > 0:
            forecast += np.dot(theta, ut[t-q:t])  # MA terms
        ut[t] = y[t] - forecast
        forecasts.append(forecast)

    forecast_df = pd.DataFrame({"Forecast": forecasts}, index=forecast_dates)
        
    return forecast_df


arch_forecast = arma_garch_forecast(arch_model_fit, test_set, 4,1,1,0)
garch_forecast = arma_garch_forecast(garch_model_fit,test_set,4,1,1,1)
plt.plot(arch_forecast, label='ARMA(4,1) - ARCH(1) forecast of yt')
plt.plot(garch_forecast, label='ARMA(4,1) - GARCH(1) forecast of yt')
plt.plot(y_forecast_df, label='ARMA(4,1) forecast of yt')
plt.plot(test_set[4:], label='yt')
plt.legend()
plt.title('Forecasted value of yt vs yt')
plt.ylabel('yt')
plt.xlabel('date')
plt.show()

### 3.g)

In [ ]:
print(((test_set[4:] - arch_forecast['Forecast']) ** 2 ).mean()) #arch
print(((test_set[4:] - garch_forecast['Forecast']) ** 2 ).mean()) #garch
print(((test_set[4:] - y_forecast_values) ** 2).mean()) #arma(4,1)
print(((test_set[4:] - test_set.shift(1).dropna()[3:]) ** 2).mean()) #naive
print((test_set[4:]**2).mean()) # forecast of 0